In [1]:
import json
import pandas as pd
import csv
from pymongo import MongoClient
import pyodbc
import pymssql 

In [58]:
try:
    client = MongoClient('mongodb://localhost:27017')
    client.server_info()
    print("Conexion exitosa")
    client.close
except pymongo.errors.ServerSelectionTimeoutError as errorTiempo:
    print("Conexión rechazada")
dbc=client['location_pasto']
colect=dbc['aborto']

tweets = []
users = {}
cols_tweet = ['id', 'created_at', 'favorite_count', 'retweet_count', 'text', 'place', 'favorited', 'retweeted']
cols_user = [
             'screen_name',
             'location',
             'followers_count',
             'statuses_count',
             'description',
             'friends_count',
             'favourites_count',
              'id',
             'name',             
             'verified']

for t in colect.find():
    tr = [t[c] for c in cols_tweet] + [t['user']['name'], t['user']['id']]
    tr.append(t['retweeted_status']['id'] if t['retweeted'] else t['id'])
    tweets.append(tr)
    uid = t['user']['id'] 
    if uid not in users:
        users[uid] = [t['user'][c] for c in cols_user]
print('Proceso realizado con éxito')

Conexion exitosa
Proceso realizado con éxito


In [59]:
tweets = pd.DataFrame(tweets, columns=cols_tweet + ['user_name', 'user_id', 'original_id'])

In [60]:
users = pd.DataFrame(users.values(), columns=cols_user)

In [83]:
africa = pd.concat([tweets,users], axis=1)

In [54]:
#datos visuales
users.head(5)

,screen_name,location,followers_count,statuses_count,description,friends_count,favourites_count,created_at,id,name,verified
0,CCirvera,Santiago,994,4631,"Orgulloso de ser Chileno, Patriota, #Chilelibr...",1404,13662,Wed May 29 02:38:43 +0000 2013,1466235415,Cristian Rodríguez,False
1,Resiliencia_51,South América,4046,12033,Yo brindo mi apoyo irrestricto e incondicional...,4159,10814,Tue Oct 06 20:20:57 +0000 2020,1313574204781088771,@paladino_1971,False
2,la_teacher2019,None,312,81942,http://Desp.de Aviones.Estud.Pedagogia.Medicin...,98,134858,Mon Mar 14 20:04:15 +0000 2011,266174763,Yone,False
3,St_Mastermind,En algún lugar de un gran país,253,47582,"Be a king? Think not, why be a king when you c...",1544,62761,Sat Aug 24 22:08:46 +0000 2013,1697564041,Master,False
4,Mariana_SN97,"Bogota, Colombia",44,159,Comunicación Social y Periodismo PUJ,637,901,Sat Aug 06 19:00:39 +0000 2011,349815416,Mariana Saldarriaga,False


In [65]:
tweets.columns

Index(['id', 'created_at', 'favorite_count', 'retweet_count', 'text', 'place',
       'favorited', 'retweeted', 'user_name', 'user_id', 'original_id'],
      dtype='object')

In [66]:
tweets.columns  = ['id', 'created_at', 'favorite_count', 'retweet_count', 'text', 'place',
       'favorited', 'retweeted', 'user_name', 'id', 'original_id']

In [67]:
tweets

,id,created_at,favorite_count,retweet_count,text,place,favorited,retweeted,user_name,id,original_id
0,1498036035590053889,Sun Feb 27 20:43:17 +0000 2022,0,0,RT @Jou_Kaiser: La ONU anda tan preocupada de ...,None,False,False,Cristian Rodríguez,1466235415,1498036035590053889
1,1498036036273901568,Sun Feb 27 20:43:17 +0000 2022,0,0,"RT @Puntodequiebr15: A esta hora en Bogotá, se...",None,False,False,@paladino_1971,1313574204781088771,1498036036273901568
2,1498036044993904648,Sun Feb 27 20:43:19 +0000 2022,0,0,RT @Jou_Kaiser: La ONU anda tan preocupada de ...,None,False,False,Yone,266174763,1498036044993904648
3,1498036049376907269,Sun Feb 27 20:43:20 +0000 2022,0,0,RT @LaVaneMG: Y dizque médico cirujano.\nQue d...,None,False,False,Master,1697564041,1498036049376907269
4,1498036055722926085,Sun Feb 27 20:43:22 +0000 2022,0,0,Me hacen falta un par de amigos provida para s...,None,False,False,Mariana Saldarriaga,349815416,1498036055722926085
...,...,...,...,...,...,...,...,...,...,...,...
10061,1498102978615652362,Mon Feb 28 01:09:18 +0000 2022,0,0,RT @PaolaHolguin: El pueblo colombiano está he...,None,False,False,Humberto Lopez Díaz,1467845475424423946,1498102978615652362
10062,1498102980398272519,Mon Feb 28 01:09:18 +0000 2022,0,0,RT @nikolas_dm: Imagina se toda mulher tivesse...,None,False,False,Rogério C Guimarães,912132396,1498102980398272519
10063,1498102989772443649,Mon Feb 28 01:09:20 +0000 2022,0,0,RT @ELTIEMPO: 🔴 #ColombiaVota2022 | Aydeé Liza...,None,False,False,Johanna Aguirre L.,2734171373,1498102989772443649
10064,1498102996168806404,Mon Feb 28 01:09:22 +0000 2022,0,0,RT @CienciaProvida: Si se despenaliza el abort...,None,False,False,La Moon,346067888,1498102996168806404


In [56]:
users.to_csv('documents/users_pasto.csv', index=False, encoding='utf8')

In [57]:
tweets.to_csv('documents/tweets_pasto.csv', index=False, encoding='utf8')

In [89]:
africa.to_csv('africa.csv', index=False, encoding='utf8')


In [71]:
output1

,id,created_at_x,favorite_count,retweet_count,text,place,favorited,retweeted,user_name,user_id,...,screen_name,location,followers_count,statuses_count,description,friends_count,favourites_count,created_at_y,name,verified
